Оценка RAG

In [ ]:
from typing import List
import os
import asyncio
import json
import numpy as np
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas import EvaluationDataset, evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, ResponseRelevancy
import os
import asyncio
from typing import Dict, Any

from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
load_dotenv(override=True)

AGENT_HOST = os.getenv("AGENT_HOST", "http://localhost")
AGENT_PORT = int(os.getenv("AGENT_PORT", 2024))
AGENT_API_KEY = os.getenv("AGENT_API_KEY")
AGENT_NAME = os.getenv("AGENT_NAME", "agent")

from langgraph_sdk import get_client 

client = get_client(url=AGENT_HOST+":"+str(AGENT_PORT), api_key=AGENT_API_KEY)

async def answer_to_user_func(context: str) -> str:
        
    assistants = await client.assistants.search()
    assistants = await client.assistants.search(graph_id=AGENT_NAME)
    
    agent = assistants[0]
    thread = await client.threads.create()
    
    user_text = context
    if not user_text:
        return "❗ Пустой запрос"
    human = HumanMessage(content=user_text)
    payload = {"messages": [human]}

    try:
        async for response_obj in client.runs.stream(thread['thread_id'], agent['assistant_id'], input=payload, config={"recursion_limit": 100}):
            print(response_obj)
        
        msgs = response_obj.data['messages'][-1]["content"]
            
        if not msgs:
            return "Ошибка при ответе, праститеееее исправимся, дайте деняк по больше токенов и сооооон"
        
        return msgs
    except Exception as e:
        return f"⚠️ Ошибка при обращении к агенту: {e}"


In [ ]:
import os
import json
import asyncio
from typing import List
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.messages import HumanMessage
from ragas import EvaluationDataset, evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, ResponseRelevancy
from dotenv import load_dotenv
from tqdm import tqdm
from typing import List
import os
import asyncio
import json
import numpy as np
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas import EvaluationDataset, evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import  Faithfulness, ResponseRelevancy,ContextPrecision, ContextRecall, AnswerCorrectness

MAX_TOKENS = 2048 * 1
from functools import lru_cache
from typing import Any, Callable, List
import json
import os
import requests

import pandas as pd
from datasets import Dataset
from tqdm import tqdm

from langchain_core.tools import tool
from react_agent.configuration import Configuration

from qdrant_client import QdrantClient, models
from fastembed import SparseTextEmbedding
from yandex_cloud_ml_sdk import YCloudML

# Qdrant + Embedding SDK
qclient = QdrantClient(os.environ["QDRANT_URL"], api_key=os.environ["QDRANT_API_KEY"])
sdk = YCloudML(folder_id=os.environ["FOLDER_ID"], auth=os.environ["API_KEY"])

dense_query_embedding_model = sdk.models.text_embeddings("query")
dense_doc_embedding_model = sdk.models.text_embeddings("doc")
bm25_embedding_model = SparseTextEmbedding("Qdrant/bm25")

model = sdk.models.completions("yandexgpt", model_version="rc")
with open('/home/ivzarru/mathmod2025/Yandex-multimodal-assistant/src/react_agent/data/tagging_prompt.md', 'r', encoding='utf-8') as f:
    TAGGING_PROMPT = f.read()

Создание датасета

In [ ]:
import os
import json
import asyncio
from tqdm import tqdm
from dotenv import load_dotenv
from langchain.schema import HumanMessage
from langchain.chat_models import ChatOpenAI
from langgraph_sdk import get_client
import qdrant_client.models as models

# Импорт функции ответов агентного API
import os
import asyncio
from typing import Dict, Any

from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
load_dotenv(override=True)

AGENT_HOST = os.getenv("AGENT_HOST", "http://localhost")
AGENT_PORT = int(os.getenv("AGENT_PORT", 2024))
AGENT_API_KEY = os.getenv("AGENT_API_KEY")
AGENT_NAME = os.getenv("AGENT_NAME", "agent")

from langgraph_sdk import get_client 

client = get_client(url=AGENT_HOST+":"+str(AGENT_PORT), api_key=AGENT_API_KEY)

async def answer_to_user_func(context: str) -> str:
        
    assistants = await client.assistants.search()
    assistants = await client.assistants.search(graph_id=AGENT_NAME)
    
    agent = assistants[0]
    thread = await client.threads.create()
    
    user_text = context
    if not user_text:
        return "❗ Пустой запрос"
    human = HumanMessage(content=user_text)
    payload = {"messages": [human]}

    try:
        async for response_obj in client.runs.stream(thread['thread_id'], agent['assistant_id'], input=payload, config={"recursion_limit": 100}):
            print(response_obj)
        
        msgs = response_obj.data['messages'][-1]["content"]
            
        if not msgs:
            return "Ошибка при ответе, праститеееее исправимся, дайте деняк по больше токенов и сооооон"
        
        return msgs
    except Exception as e:
        return f"⚠️ Ошибка при обращении к агенту: {e}"

# Импорт для оценки
from yandex_chain import YandexEmbeddings


# Загрузка переменных окружения
load_dotenv(override=True)

# ----- Настройка LLM для тегирования и эмбеддингов -----
llm = ChatOpenAI(
    model="gpt://" + os.environ["FOLDER_ID"] + '/yandexgpt/rc',
    api_key=os.environ["API_KEY"],
    temperature=0.1,
    base_url="https://llm.api.cloud.yandex.net/v1"
)

# === Инициализируйте свои модели эмбеддингов и Qdrant клиент ===
# dense_query_embedding_model = ...
# bm25_embedding_model = ...
# qclient = ...
# TAGGING_PROMPT = "..."
# MAX_TOKENS = 1024

# Функция для тегирования запроса
def tag_query(query: str) -> str:
    return llm.invoke(input=TAGGING_PROMPT + "\n\n Входной запрос:\n" + query).content

# Функция поиска по коллекции Qdrant
def query_from_collection(query: str, collection_name: str, top_k: int = 5) -> str:
    dense_query_vector = dense_query_embedding_model.run(query[:MAX_TOKENS*2]).embedding
    sparse_query_vector = list(bm25_embedding_model.embed([query]))[0]
    query_topics = set(tag_query(query).split(', '))
    query_topics.discard('мусор')

    prefetch = [
        models.Prefetch(query=dense_query_vector, using="dense", limit=6*top_k),
        models.Prefetch(query=models.SparseVector(**sparse_query_vector.as_object()), using="sparse", limit=6*top_k),
    ]

    resp = qclient.query_points(
        collection_name,
        prefetch=prefetch,
        query=models.FusionQuery(fusion=models.Fusion.RRF),
        with_payload=True,
        limit=7*top_k,
    )
    threshold = sum(p.score for p in resp.points) / len(resp.points)

    results = []
    for point in resp.points:
        source = point.payload['source']
        if 'chat' in source:
            date = source.split('_')[0].replace('chat', '')
        else:
            date = {
                'Онлайн_магистратура_«Машинное_обучение_и_анализ_данных»': 2024,
                'Особые_условия_для_поступления_в_Институт_№8_МАИ': '2024-2026',
                'Постановление Правительства РФ от 27.04.2024 N 555 — Редакция от 07.04.2025 — Контур.Норматив': '2025-2026',
                'Правила приема МАИ': '2025-2026',
                'Правила Приема(Министерские)': '2024 - 2026',
                'Федеральный закон от 29 декабря 2012 г. N 273-ФЗ _Об образовании в Российской Фе ... _ Система ГАРАНТ': '2012 - 2025'
            }.get(source, '2025-2026')

        if set(point.payload['topics']).intersection(query_topics) or point.score > threshold:
            results.append(f"Актуально в период: {date}\n\n{point.payload['chunk_text']}")

    results = results[:top_k]
    return "\n".join([
        "=" * 20 + f"\n# Источник номер {i + 1}\n" + text
        for i, text in enumerate(results)
    ])

# Функция получения списком отрывков контекста
def retrieve(query: str, top_k: int = 10) -> list[str]:
    raw = query_from_collection(query, "Collection_pdf", top_k=top_k)
    parts = [p.strip() for p in raw.split("=" * 20) if p.strip()]
    return parts[:top_k]

# Главная асинхронная функция
async def main():
    # Чтение входных данных
    questions = open('../data/testing_data/test_cases.txt', encoding='utf-8').read().splitlines()
    gt = open('../data/testing_data/ground_truth.txt', 'r', encoding='utf-8').read().splitlines('$$$')

    dataset = []
    mew = []
    # Загрузка уже существующих ответов
    try:
        with open('mew.json', 'r', encoding='utf-8') as f:
            mew = json.load(f)
    except FileNotFoundError:
        mew = []

    c = 0
    for q in tqdm(questions, desc="Обработка вопросов", unit="вопрос"):
        # Проверяем, есть ли уже такой вопрос
        existing = next((e for e in mew if e['user_input'] == q), None)
        if existing:
            existing['reference'] = gt[c]
            dataset.append(existing)
            c += 1
            continue

        # Получаем контекст и формируем вход для агента
        contexts = retrieve(q)
        prompt = f"""

🎓 **AI-ассистент приёмной комиссии Московского авиационного института (МАИ)**

🧑‍💼 **Твоя роль**  
Ты — умный, доброжелательный и чёткий AI-помощник.  
Твоя задача — сделать процесс поступления в МАИ **максимально простым и понятным** для каждого абитуриента.

💬 **Как ты общаешься**
- **Просто** — как будто объясняешь другу.
- **По делу** — ничего лишнего, только нужная информация.
- **С заботой** — чтобы поступающий чувствовал поддержку.

📌 **Пример ответа:**  
`Чтобы подать документы в МАИ онлайн, зайдите в личный кабинет на сайте приёмной комиссии. Вам понадобятся сканы паспорта, аттестата и СНИЛС.`

🔍 **Как ты работаешь (логика диалога)**  
Твой принцип: **думай → ищи → анализируй → помогай → проверяй → уточняй**

1. **Каждый новый вопрос = новая гипотеза**
    - Определи, чего хочет абитуриент.
    - Понял? Сделай уточнение — даже если уверен.
    - Не понял? Мягко уточни.

2. **Всегда запускай ИНСТРУМЕНТЫ:**
    - Сначала: `search` и `yandex_generative_search` (одновременно).
    - Потом, по ситуации, повтори вызовы с уточнённым запросом.
    - Меняй формулировки, ищи другие форматы ответа.
    - Можно вызывать инструменты несколько раз — это правильно.

3. **Обязательно АНАЛИЗИРУЙ каждый результат:**
    - Что из найденного реально помогает?  
    - Есть ли в нём дедлайны, баллы, нюансы?

4. **Формируй ответ просто и точно**
    - Со ссылками или с прямым действием.
    - Если есть дедлайны — предупреди.
    - Если есть опасность ошибки — подчеркни её.
    - Если вопрос сложный — веди пошагово: сначала одно, потом следующее.

5. **Если после 2–3 поисков нет результата — не тяни:**
    - Предложи обратиться к оператору:
    `По этому вопросу лучше уточнить в приёмной комиссии МАИ напрямую.`

📌 **Всегда уточняй:**
- Бакалавриат или магистратура?
- Российский или иностранный абитуриент?
- На платное или бюджетное?

📅 **Контекст времени**
— Помни про актуальность данных. Уточняй год, если вопрос может касаться прошлых лет.  
— Учитывай, что информация может меняться — проверяй через инструменты.


📥 **Онлайн-подача**
Через Госуслуги или личный кабинет МАИ. Нужны: паспорт, аттестат, СНИЛС.  

📢 **Согласие на зачисление**
Не забудь: приоритет за теми, кто подал оригинал и согласие.

🧠 **Твои главные принципы**
✅ **Ты ведёшь диалог до результата**  
✅ **Ты защищаешь абитуриента от ошибок**  
✅ **Ты делаешь сложное простым**
✅ **Будь проактивен. Предлагай, веди беседу**

📞 **Что делать, если не знаешь ответа?**
1. Сначала — **поиск (search + yandex_generative_search)**  
2. Потом — **анализируй** и снова ищи, если нужно.  
3. Только если совсем нет данных — скажи:  
`К сожалению, сейчас я не могу найти точной информации. Рекомендую связаться с приёмной комиссией МАИ напрямую.`

🎯 **Твоя миссия**
Сделать так, чтобы любой человек — независимо от подготовки — мог поступить в МАИ спокойно и уверенно.  
Ты — как личный наставник, который всегда рядом.


**ОТВЕЧАЙ ПОЛЬЗОВАТЕЛЮ НА ЯЗЫКЕ, КОТОРЫЙ ОН ИСПОЛЬЗУЕТ. **

Запрос пользователя:{q}
"""

        # Получаем ответ от агентного API
        response_text = await answer_to_user_func(prompt)
        if not response_text:
            response_text = "❗ Агент вернул пустой ответ"

        entry = {
            "user_input": q,
            "retrieved_contexts": contexts,
            "reference": gt[c],
            "response": response_text
        }
        c += 1
        dataset.append(entry)
        mew.append(entry)

        # Сохраняем промежуточный файл
        with open('mew.json', 'w', encoding='utf-8') as f:
            json.dump(mew, f, ensure_ascii=False, indent=4)

    # Сохраняем полный датасет
    with open('dataset.json', 'w', encoding='utf-8') as f:
        json.dump(dataset, f, ensure_ascii=False, indent=4)

    # Подготавливаем к оценке
    eval_dataset = EvaluationDataset.from_list(dataset)

    evaluator_llm = ChatOpenAI(
        model="gpt://" + os.environ["FOLDER_ID"] + '/yandexgpt/rc',
        api_key=os.environ["API_KEY"],
        temperature=0,
        base_url="https://llm.api.cloud.yandex.net/v1"
    )
    embeddings = YandexEmbeddings(
        api_key=os.environ["API_KEY"],
        folder_id=os.environ["FOLDER_ID"],
        model_version="rc",
        retries=10
    )
    evaluator = LangchainLLMWrapper(evaluator_llm)

    metrics = [AnswerCorrectness()]

    # Оценка ответов агента
    results = evaluate(dataset=eval_dataset, metrics=metrics, llm=evaluator, embeddings=embeddings)
    print(results)

if __name__ == "__main__":
    asyncio.run(main())


In [ ]:
from ragas import EvaluationDataset
dataset = json.load(open('dataset.json', 'r', encoding='utf-8'))
eval_ds = EvaluationDataset.from_list(dataset)


Оценка

In [ ]:
os.environ["API_KEY"]

In [ ]:
! pip install yandex-chain

In [ ]:
from yandex_chain import YandexEmbeddings

embeddings = YandexEmbeddings(
    api_key=os.environ["API_KEY"],
    folder_id=os.environ["FOLDER_ID"],
    model_version="rc"
)


embeddings.embed_documents(["Привет, мир!"])

In [ ]:
from ragas.metrics import  Faithfulness, ResponseRelevancy
from dotenv import load_dotenv
import json
import shutil  # Импортируем shutil для очистки папки

# Загрузка переменных окружения
load_dotenv()
with open('dataset.json', 'r', encoding='utf-8') as f:
    dataset = json.load(f)
eval_dataset = EvaluationDataset.from_list(dataset)
llm = ChatOpenAI(
    model="gpt://" + os.environ["FOLDER_ID"] + '/yandexgpt/rc',
    api_key=os.environ["API_KEY"],
    temperature=0.1,
    base_url="https://llm.api.cloud.yandex.net/v1"
)
evaluator = LangchainLLMWrapper(llm)
metrics = [
    Faithfulness(),
    ResponseRelevancy()
]
results = evaluate(
    dataset=eval_dataset,
    metrics=metrics,
    llm=evaluator,
    embeddings=embeddings
)
print(results)

In [ ]:
llm = ChatOpenAI(
    model="gpt://" + os.environ["FOLDER_ID"] + '/yandexgpt/rc',
    api_key=os.environ["API_KEY"],
    temperature=0.1,
    base_url="https://llm.api.cloud.yandex.net/v1"
)

In [ ]:
llm.invoke("fdfd")